In [1]:
!pip install pycolmap
!apt-get update
!apt-get install -y ffmpeg
!pip install git+https://github.com/rahul-goel/fused-ssim/ --no-build-isolation

import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/camera_photos/gaussian-splatting/submodules/")
!git clone https://huggingface.co/spaces/ashawkey/LGM
os.system('pip install -e ./diff-gaussian-rasterization')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 37.6 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,267 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,157 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,124 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubun

256

In [2]:
# read_write_model.py
import argparse
import collections
import os
import struct
import numpy as np

CameraModel = collections.namedtuple(
    "CameraModel", ["model_id", "model_name", "num_params"]
)
Camera = collections.namedtuple(
    "Camera", ["id", "model", "width", "height", "params"]
)
BaseImage = collections.namedtuple(
    "Image", ["id", "qvec", "tvec", "camera_id", "name", "xys", "point3D_ids"]
)
Point3D = collections.namedtuple(
    "Point3D", ["id", "xyz", "rgb", "error", "image_ids", "point2D_idxs"]
)


class Image(BaseImage):
    def qvec2rotmat(self):
        return qvec2rotmat(self.qvec)


CAMERA_MODELS = {
    CameraModel(model_id=0, model_name="SIMPLE_PINHOLE", num_params=3),
    CameraModel(model_id=1, model_name="PINHOLE", num_params=4),
    CameraModel(model_id=2, model_name="SIMPLE_RADIAL", num_params=4),
    CameraModel(model_id=3, model_name="RADIAL", num_params=5),
    CameraModel(model_id=4, model_name="OPENCV", num_params=8),
    CameraModel(model_id=5, model_name="OPENCV_FISHEYE", num_params=8),
    CameraModel(model_id=6, model_name="FULL_OPENCV", num_params=12),
    CameraModel(model_id=7, model_name="FOV", num_params=5),
    CameraModel(model_id=8, model_name="SIMPLE_RADIAL_FISHEYE", num_params=4),
    CameraModel(model_id=9, model_name="RADIAL_FISHEYE", num_params=5),
    CameraModel(model_id=10, model_name="THIN_PRISM_FISHEYE", num_params=12),
}
CAMERA_MODEL_IDS = dict(
    [(camera_model.model_id, camera_model) for camera_model in CAMERA_MODELS]
)
CAMERA_MODEL_NAMES = dict(
    [(camera_model.model_name, camera_model) for camera_model in CAMERA_MODELS]
)


def read_next_bytes(fid, num_bytes, format_char_sequence, endian_character="<"):
    """Read and unpack the next bytes from a binary file.
    :param fid:
    :param num_bytes: Sum of combination of {2, 4, 8}, e.g. 2, 6, 16, 30, etc.
    :param format_char_sequence: List of {c, e, f, d, h, H, i, I, l, L, q, Q}.
    :param endian_character: Any of {@, =, <, >, !}
    :return: Tuple of read and unpacked values.
    """
    data = fid.read(num_bytes)
    return struct.unpack(endian_character + format_char_sequence, data)


def write_next_bytes(fid, data, format_char_sequence, endian_character="<"):
    """pack and write to a binary file.
    :param fid:
    :param data: data to send, if multiple elements are sent at the same time,
    they should be encapsuled either in a list or a tuple
    :param format_char_sequence: List of {c, e, f, d, h, H, i, I, l, L, q, Q}.
    should be the same length as the data list or tuple
    :param endian_character: Any of {@, =, <, >, !}
    """
    if isinstance(data, (list, tuple)):
        bytes = struct.pack(endian_character + format_char_sequence, *data)
    else:
        bytes = struct.pack(endian_character + format_char_sequence, data)
    fid.write(bytes)


def read_cameras_text(path):
    """
    see: src/colmap/scene/reconstruction.cc
        void Reconstruction::WriteCamerasText(const std::string& path)
        void Reconstruction::ReadCamerasText(const std::string& path)
    """
    cameras = {}
    with open(path, "r") as fid:
        while True:
            line = fid.readline()
            if not line:
                break
            line = line.strip()
            if len(line) > 0 and line[0] != "#":
                elems = line.split()
                camera_id = int(elems[0])
                model = elems[1]
                width = int(elems[2])
                height = int(elems[3])
                params = np.array(tuple(map(float, elems[4:])))
                cameras[camera_id] = Camera(
                    id=camera_id,
                    model=model,
                    width=width,
                    height=height,
                    params=params,
                )
    return cameras


def read_cameras_binary(path_to_model_file):
    cameras = {}
    with open(path_to_model_file, "rb") as fid:
        num_cameras = read_next_bytes(fid, 8, "Q")[0]
        for _ in range(num_cameras):
            camera_properties = read_next_bytes(
                fid, num_bytes=24, format_char_sequence="iiQQ"
            )
            camera_id = camera_properties[0]
            model_id = camera_properties[1]
            model_name = CAMERA_MODEL_IDS[camera_properties[1]].model_name
            width = camera_properties[2]
            height = camera_properties[3]
            num_params = CAMERA_MODEL_IDS[model_id].num_params
            params = read_next_bytes(
                fid,
                num_bytes=8 * num_params,
                format_char_sequence="d" * num_params,
            )
            cameras[camera_id] = Camera(
                id=camera_id,
                model=model_name,
                width=width,
                height=height,
                params=np.array(params),
            )
        assert len(cameras) == num_cameras
    return cameras


def write_cameras_text(cameras, path):
    HEADER = (
        "# Camera list with one line of data per camera:\n"
        + "#   CAMERA_ID, MODEL, WIDTH, HEIGHT, PARAMS[]\n"
        + "# Number of cameras: {}\n".format(len(cameras))
    )
    with open(path, "w") as fid:
        fid.write(HEADER)
        for _, cam in cameras.items():
            to_write = [cam.id, cam.model, cam.width, cam.height, *cam.params]
            line = " ".join([str(elem) for elem in to_write])
            fid.write(line + "\n")


def write_cameras_binary(cameras, path_to_model_file):
    with open(path_to_model_file, "wb") as fid:
        write_next_bytes(fid, len(cameras), "Q")
        for _, cam in cameras.items():
            model_id = CAMERA_MODEL_NAMES[cam.model].model_id
            camera_properties = [cam.id, model_id, cam.width, cam.height]
            write_next_bytes(fid, camera_properties, "iiQQ")
            for p in cam.params:
                write_next_bytes(fid, float(p), "d")
    return cameras


def read_images_text(path):
    images = {}
    with open(path, "r") as fid:
        while True:
            line = fid.readline()
            if not line:
                break
            line = line.strip()
            if len(line) > 0 and line[0] != "#":
                elems = line.split()
                image_id = int(elems[0])
                qvec = np.array(tuple(map(float, elems[1:5])))
                tvec = np.array(tuple(map(float, elems[5:8])))
                camera_id = int(elems[8])
                image_name = elems[9]
                elems = fid.readline().split()
                xys = np.column_stack(
                    [
                        tuple(map(float, elems[0::3])),
                        tuple(map(float, elems[1::3])),
                    ]
                )
                point3D_ids = np.array(tuple(map(int, elems[2::3])))
                images[image_id] = Image(
                    id=image_id,
                    qvec=qvec,
                    tvec=tvec,
                    camera_id=camera_id,
                    name=image_name,
                    xys=xys,
                    point3D_ids=point3D_ids,
                )
    return images


def read_images_binary(path_to_model_file):
    images = {}
    with open(path_to_model_file, "rb") as fid:
        num_reg_images = read_next_bytes(fid, 8, "Q")[0]
        for _ in range(num_reg_images):
            binary_image_properties = read_next_bytes(
                fid, num_bytes=64, format_char_sequence="idddddddi"
            )
            image_id = binary_image_properties[0]
            qvec = np.array(binary_image_properties[1:5])
            tvec = np.array(binary_image_properties[5:8])
            camera_id = binary_image_properties[8]
            binary_image_name = b""
            current_char = read_next_bytes(fid, 1, "c")[0]
            while current_char != b"\x00":  # look for the ASCII 0 entry
                binary_image_name += current_char
                current_char = read_next_bytes(fid, 1, "c")[0]
            image_name = binary_image_name.decode("utf-8")
            num_points2D = read_next_bytes(
                fid, num_bytes=8, format_char_sequence="Q"
            )[0]
            x_y_id_s = read_next_bytes(
                fid,
                num_bytes=24 * num_points2D,
                format_char_sequence="ddq" * num_points2D,
            )
            xys = np.column_stack(
                [
                    tuple(map(float, x_y_id_s[0::3])),
                    tuple(map(float, x_y_id_s[1::3])),
                ]
            )
            point3D_ids = np.array(tuple(map(int, x_y_id_s[2::3])))
            images[image_id] = Image(
                id=image_id,
                qvec=qvec,
                tvec=tvec,
                camera_id=camera_id,
                name=image_name,
                xys=xys,
                point3D_ids=point3D_ids,
            )
    return images


def write_images_text(images, path):
    if len(images) == 0:
        mean_observations = 0
    else:
        mean_observations = sum(
            (len(img.point3D_ids) for _, img in images.items())
        ) / len(images)
    HEADER = (
        "# Image list with two lines of data per image:\n"
        + "#   IMAGE_ID, QW, QX, QY, QZ, TX, TY, TZ, CAMERA_ID, NAME\n"
        + "#   POINTS2D[] as (X, Y, POINT3D_ID)\n"
        + "# Number of images: {}, mean observations per image: {}\n".format(
            len(images), mean_observations
        )
    )

    with open(path, "w") as fid:
        fid.write(HEADER)
        for _, img in images.items():
            image_header = [
                img.id,
                *img.qvec,
                *img.tvec,
                img.camera_id,
                img.name,
            ]
            first_line = " ".join(map(str, image_header))
            fid.write(first_line + "\n")

            points_strings = []
            for xy, point3D_id in zip(img.xys, img.point3D_ids):
                points_strings.append(" ".join(map(str, [*xy, point3D_id])))
            fid.write(" ".join(points_strings) + "\n")


def write_images_binary(images, path_to_model_file):
    with open(path_to_model_file, "wb") as fid:
        write_next_bytes(fid, len(images), "Q")
        for _, img in images.items():
            write_next_bytes(fid, img.id, "i")
            write_next_bytes(fid, img.qvec.tolist(), "dddd")
            write_next_bytes(fid, img.tvec.tolist(), "ddd")
            write_next_bytes(fid, img.camera_id, "i")
            for char in img.name:
                write_next_bytes(fid, char.encode("utf-8"), "c")
            write_next_bytes(fid, b"\x00", "c")
            write_next_bytes(fid, len(img.point3D_ids), "Q")
            for xy, p3d_id in zip(img.xys, img.point3D_ids):
                write_next_bytes(fid, [*xy, p3d_id], "ddq")


def read_points3D_text(path):
    points3D = {}
    with open(path, "r") as fid:
        while True:
            line = fid.readline()
            if not line:
                break
            line = line.strip()
            if len(line) > 0 and line[0] != "#":
                elems = line.split()
                point3D_id = int(elems[0])
                xyz = np.array(tuple(map(float, elems[1:4])))
                rgb = np.array(tuple(map(int, elems[4:7])))
                error = float(elems[7])
                image_ids = np.array(tuple(map(int, elems[8::2])))
                point2D_idxs = np.array(tuple(map(int, elems[9::2])))
                points3D[point3D_id] = Point3D(
                    id=point3D_id,
                    xyz=xyz,
                    rgb=rgb,
                    error=error,
                    image_ids=image_ids,
                    point2D_idxs=point2D_idxs,
                )
    return points3D


def read_points3D_binary(path_to_model_file):
    points3D = {}
    with open(path_to_model_file, "rb") as fid:
        num_points = read_next_bytes(fid, 8, "Q")[0]
        for _ in range(num_points):
            binary_point_line_properties = read_next_bytes(
                fid, num_bytes=43, format_char_sequence="QdddBBBd"
            )
            point3D_id = binary_point_line_properties[0]
            xyz = np.array(binary_point_line_properties[1:4])
            rgb = np.array(binary_point_line_properties[4:7])
            error = np.array(binary_point_line_properties[7])
            track_length = read_next_bytes(
                fid, num_bytes=8, format_char_sequence="Q"
            )[0]
            track_elems = read_next_bytes(
                fid,
                num_bytes=8 * track_length,
                format_char_sequence="ii" * track_length,
            )
            image_ids = np.array(tuple(map(int, track_elems[0::2])))
            point2D_idxs = np.array(tuple(map(int, track_elems[1::2])))
            points3D[point3D_id] = Point3D(
                id=point3D_id,
                xyz=xyz,
                rgb=rgb,
                error=error,
                image_ids=image_ids,
                point2D_idxs=point2D_idxs,
            )
    return points3D


def write_points3D_text(points3D, path):
    """
    see: src/colmap/scene/reconstruction.cc
        void Reconstruction::ReadPoints3DText(const std::string& path)
        void Reconstruction::WritePoints3DText(const std::string& path)
    """
    if len(points3D) == 0:
        mean_track_length = 0
    else:
        mean_track_length = sum(
            (len(pt.image_ids) for _, pt in points3D.items())
        ) / len(points3D)
    HEADER = (
        "# 3D point list with one line of data per point:\n"
        + "#   POINT3D_ID, X, Y, Z, R, G, B, ERROR, TRACK[] as (IMAGE_ID, POINT2D_IDX)\n"
        + "# Number of points: {}, mean track length: {}\n".format(
            len(points3D), mean_track_length
        )
    )

    with open(path, "w") as fid:
        fid.write(HEADER)
        for _, pt in points3D.items():
            point_header = [pt.id, *pt.xyz, *pt.rgb, pt.error]
            fid.write(" ".join(map(str, point_header)) + " ")
            track_strings = []
            for image_id, point2D in zip(pt.image_ids, pt.point2D_idxs):
                track_strings.append(" ".join(map(str, [image_id, point2D])))
            fid.write(" ".join(track_strings) + "\n")


def write_points3D_binary(points3D, path_to_model_file):
    """
    see: src/colmap/scene/reconstruction.cc
        void Reconstruction::ReadPoints3DBinary(const std::string& path)
        void Reconstruction::WritePoints3DBinary(const std::string& path)
    """
    with open(path_to_model_file, "wb") as fid:
        write_next_bytes(fid, len(points3D), "Q")
        for _, pt in points3D.items():
            write_next_bytes(fid, pt.id, "Q")
            write_next_bytes(fid, pt.xyz.tolist(), "ddd")
            write_next_bytes(fid, pt.rgb.tolist(), "BBB")
            write_next_bytes(fid, pt.error, "d")
            track_length = pt.image_ids.shape[0]
            write_next_bytes(fid, track_length, "Q")
            for image_id, point2D_id in zip(pt.image_ids, pt.point2D_idxs):
                write_next_bytes(fid, [image_id, point2D_id], "ii")


def detect_model_format(path, ext):
    if (
        os.path.isfile(os.path.join(path, "cameras" + ext))
        and os.path.isfile(os.path.join(path, "images" + ext))
        and os.path.isfile(os.path.join(path, "points3D" + ext))
    ):
        print("Detected model format: '" + ext + "'")
        return True
    return False


def read_model(path, ext=""):
    # try to detect the extension automatically
    if ext == "":
        if detect_model_format(path, ".bin"):
            ext = ".bin"
        elif detect_model_format(path, ".txt"):
            ext = ".txt"
        else:
            print("Provide model format: '.bin' or '.txt'")
            return

    if ext == ".txt":
        cameras = read_cameras_text(os.path.join(path, "cameras" + ext))
        images = read_images_text(os.path.join(path, "images" + ext))
        points3D = read_points3D_text(os.path.join(path, "points3D") + ext)
    else:
        cameras = read_cameras_binary(os.path.join(path, "cameras" + ext))
        images = read_images_binary(os.path.join(path, "images" + ext))
        points3D = read_points3D_binary(os.path.join(path, "points3D") + ext)
    return cameras, images, points3D


def write_model(cameras, images, points3D, path, ext=".bin"):
    if ext == ".txt":
        write_cameras_text(cameras, os.path.join(path, "cameras" + ext))
        write_images_text(images, os.path.join(path, "images" + ext))
        write_points3D_text(points3D, os.path.join(path, "points3D") + ext)
    else:
        write_cameras_binary(cameras, os.path.join(path, "cameras" + ext))
        write_images_binary(images, os.path.join(path, "images" + ext))
        write_points3D_binary(points3D, os.path.join(path, "points3D") + ext)
    return cameras, images, points3D


def qvec2rotmat(qvec):
    return np.array(
        [
            [
                1 - 2 * qvec[2] ** 2 - 2 * qvec[3] ** 2,
                2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
                2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2],
            ],
            [
                2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
                1 - 2 * qvec[1] ** 2 - 2 * qvec[3] ** 2,
                2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1],
            ],
            [
                2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
                2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
                1 - 2 * qvec[1] ** 2 - 2 * qvec[2] ** 2,
            ],
        ]
    )


def rotmat2qvec(R):
    Rxx, Ryx, Rzx, Rxy, Ryy, Rzy, Rxz, Ryz, Rzz = R.flat
    K = (
        np.array(
            [
                [Rxx - Ryy - Rzz, 0, 0, 0],
                [Ryx + Rxy, Ryy - Rxx - Rzz, 0, 0],
                [Rzx + Rxz, Rzy + Ryz, Rzz - Rxx - Ryy, 0],
                [Ryz - Rzy, Rzx - Rxz, Rxy - Ryx, Rxx + Ryy + Rzz],
            ]
        )
        / 3.0
    )
    eigvals, eigvecs = np.linalg.eigh(K)
    qvec = eigvecs[[3, 0, 1, 2], np.argmax(eigvals)]
    if qvec[0] < 0:
        qvec *= -1
    return qvec


def add_cameras_to_dict(cameras, camera_params_list):
    """
    cameras: 기존 카메라 딕셔너리
    camera_params_list: [
        {'id': 1, 'model': 'PINHOLE', 'width': 4032, 'height': 3024, 'params': [fx1, fy1, cx1, cy1]},
        {'id': 2, 'model': 'PINHOLE', 'width': 4032, 'height': 3024, 'params': [fx2, fy2, cx2, cy2]},
        ...
    ]
    """
    for cam in camera_params_list:
        cameras[cam['id']] = Camera(
            id=cam['id'],
            model=cam['model'],
            width=cam['width'],
            height=cam['height'],
            params=np.array(cam['params'])
        )
    return cameras


def main():
    parser = argparse.ArgumentParser(
        description="Read and write COLMAP binary and text models"
    )
    parser.add_argument("--input_model", help="path to input model folder")
    parser.add_argument(
        "--input_format",
        choices=[".bin", ".txt"],
        help="input model format",
        default="",
    )
    parser.add_argument("--output_model", help="path to output model folder")
    parser.add_argument(
        "--output_format",
        choices=[".bin", ".txt"],
        help="output model format",
        default=".txt",
    )
    args, _ = parser.parse_known_args()

    input_model_path = args.input_model if args.input_model else "/content/drive/MyDrive/camera_photos/happy_noodles/frames/sparse/"
    output_model_path = args.output_model if args.output_model else "/content/drive/MyDrive/camera_photos/happy_noodles/frames/sparse/"

    cameras, images, points3D = read_model(
        path=input_model_path, ext=args.input_format
    )

    camera_params_list = [       # [fx1, fy1, cx1, cy1]
        # {'id': 10, 'model': 'PINHOLE', 'width': 1209, 'height': 907, 'params': [900.32, 900.57, 604.5, 435.5]}, # 사진
        {'id': 11, 'model': 'PINHOLE', 'width': 768, 'height': 432, 'params': [571.91, 428.94, 384., 216.]},    # 동영상 1x
        # {'id': 12, 'model': 'PINHOLE', 'width': 768, 'height': 432, 'params': [209.44, 157.26, 384., 216.]},    # 동영상 0.5x
    ]
    cameras = add_cameras_to_dict(cameras, camera_params_list)

    for img in images.values():
        if img.name.startswith("5276"):
            img = img._replace(camera_id=11)
            images[img.id] = img

    print("num_cameras:", len(cameras))
    print("num_images:", len(images))
    print("num_points3D:", len(points3D))

    if output_model_path is not None:
        write_model(
            cameras,
            images,
            points3D,
            path=output_model_path,
            ext=args.output_format,
        )

main()

Detected model format: '.bin'
num_cameras: 336
num_images: 336
num_points3D: 49554


In [ ]:
# # dependencies for gaussian splatting
# !pip install joblib
# !pip install torchaudio
# !pip install plyfile

In [4]:
import os
import shutil
import pycolmap
import subprocess
from tqdm import tqdm

colmap_path = "colmap"
gpu = True

def run_pycolmap_sfm(image_dir, output_dir, num_threads=8, GPU=gpu, colmap_path="colmap"):
    """
    pycolmap의 공식 파이프라인: feature extraction, matching, SfM을 순차적으로 실행
    """
    db_path = os.path.join(output_dir, "database.db")
    sparse_dir = os.path.join(output_dir, "sparse/")
    dense_dir = os.path.join(output_dir, "dense/")
    os.makedirs(sparse_dir, exist_ok=True)
    os.makedirs(dense_dir, exist_ok=True)

    if GPU:
        matching_options = pycolmap.SequentialMatchingOptions(num_threads=num_threads)
        mapper_options = pycolmap.IncrementalPipelineOptions(num_threads=num_threads)
        steps = [
        ("Feature extraction", lambda: pycolmap.extract_features(db_path, image_dir, sift_options={"num_threads":num_threads,
                                                                                                   "use_gpu":1,
                                                                                                   "max_num_features":16000,
                                                                                                   "estimate_affine_shape":True,
                                                                                                   "domain_size_pooling":True})),
        # ("Feature matching (exhaustive)", lambda: pycolmap.match_exhaustive(db_path)),
        ("Feature matching (sequential)", lambda: pycolmap.match_sequential(db_path, matching_options=matching_options)),
        ("Incremental mapping", lambda: pycolmap.incremental_mapping(db_path, image_dir, sparse_dir, mapper_options)),
    ]
    else:
        steps = [
        ("Feature extraction", lambda: pycolmap.extract_features(db_path, image_dir)),
        ("Feature matching (exhaustive)", lambda: pycolmap.match_exhaustive(db_path)),
        # ("Feature matching (sequential)", lambda: pycolmap.match_sequential(db_path)),
        ("Incremental mapping", lambda: pycolmap.incremental_mapping(db_path, image_dir, sparse_dir)),
    ]

    results = None

    for desc, func in tqdm(steps, desc="pycolmap SfM pipeline", unit="step"):
        tqdm.write(f"실행: {desc}")
        if desc == "Incremental mapping":
            results = func()
        else:
            func()

    if results:
        tqdm.write("모델 저장...")
        results[0].write(sparse_dir)        # maps[0]이 가장 큰 모델
    tqdm.write("pycolmap SfM 완료")


def prepare_gaussian_splatting_dataset(pycolmap_sparse_dir, images_dir, output_dir):
    os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)
    imgs_dir = [x for x in os.listdir(images_dir) if x.endswith(".jpg") or x.endswith(".JPG")]

    os.makedirs(os.path.join(output_dir, "sparse/"), exist_ok=True)
    sparse_files = ["cameras.bin", "images.bin", "points3D.bin"]

def train_gaussian_splatting(dataset_dir, gs_code_dir):
    tqdm.write("Gaussian Splatting...")
    subprocess.run([
        "python", os.path.join(gs_code_dir, "train.py"),
        "-s", dataset_dir
    ])
    tqdm.write("Gaussian Splatting training 완료")

if __name__ == "__main__":
    image_dir = "/content/drive/MyDrive/camera_photos/happy_noodles/frames/images/"  # 이미지 폴더
    pycolmap_output_dir = "/content/drive/MyDrive/camera_photos/happy_noodles/frames/"
    gs_dataset_dir = pycolmap_output_dir
    gs_code_dir = "/content/drive/MyDrive/camera_photos/gaussian-splatting/"

    gpu = True
    run_pycolmap_sfm(image_dir, pycolmap_output_dir, 8, gpu, colmap_path)
    prepare_gaussian_splatting_dataset(
        pycolmap_sparse_dir=os.path.join(pycolmap_output_dir, "sparse/"),
        images_dir=image_dir,
        output_dir=gs_dataset_dir
    )
    train_gaussian_splatting(gs_dataset_dir, gs_code_dir)

pycolmap SfM pipeline:   0%|          | 0/3 [00:00<?, ?step/s]

실행: Feature extraction


pycolmap SfM pipeline:  33%|███▎      | 1/3 [12:50<25:41, 770.90s/step]

실행: Feature matching (sequential)


pycolmap SfM pipeline:  67%|██████▋   | 2/3 [15:51<07:03, 423.55s/step]

실행: Incremental mapping


pycolmap SfM pipeline: 100%|██████████| 3/3 [25:02<00:00, 500.88s/step]


모델 저장...
pycolmap SfM 완료
Gaussian Splatting...
Gaussian Splatting training 완료


In [5]:
# COLMAP의 .bin 파일을 직접 읽어 3D 포인트 클라우드와 카메라 위치를 웹 기반 3D 뷰어로 시각화해줌
!pip install viser

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.0/709.0 kB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.1/541.1 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 27.1 MB/s eta 0:00:00


In [8]:
cameras_bin = "/content/drive/MyDrive/camera_photos/happy_noodles/frames/sparse/cameras.bin"
images_bin = "/content/drive/MyDrive/camera_photos/happy_noodles/frames/sparse/images.bin"
points3D_bin = "/content/drive/MyDrive/camera_photos/happy_noodles/frames/sparse/points3D.bin"

In [9]:
# 결과물인 .bin 파일들 보여주는거
# 이거는 내 컴퓨터 로컬상에서 해야하는 거 ㅇㅇ. 코랩에서는 아무리 눌러도 안됨
import numpy as np
from pathlib import Path
import viser
from viser.extras.colmap import (read_cameras_binary, read_images_binary, read_points3d_binary)

server = viser.ViserServer()

# colmap 데이터 읽기, cameras.bin, images.bin, points3D.bin 폴더
cameras = read_cameras_binary(cameras_bin)
images = read_images_binary(images_bin)
points3d = read_points3d_binary(points3D_bin)

points = np.array([points3d[p_id].xyz for p_id in points3d])
colors = np.array([points3d[p_id].rgb for p_id in points3d])

server.scene.add_point_cloud(
    name="/colmap/pcd",
    points=points,
    colors=colors / 255.0,  # RGB 값 정규화
    point_size=0.02,
)

for img in images.values():
    server.scene.add_frame(
        f"/colmap/frame_{img.id}",
        wxyz=img.qvec,  # 쿼터니언, 4개: QX QW QY QZ (world -> camera)
        position=img.tvec, # translation, 3개: TX TY TZ,        <- 7개 합쳐서 extrinsic
        axes_length=0.1,
        axes_radius=0.005,
    )

print("Viser 서버 시작, 브라우저 localhost:8080")
print("서버를 중지하려면 Ctrl+C를 누르세요.")

╭──────────────── viser ────────────────╮
│             ╷                         │
│   HTTP      │ http://localhost:8082   │
│   Websocket │ ws://localhost:8082     │
│             ╵                         │
╰───────────────────────────────────────╯

Viser 서버 시작, 브라우저 localhost:8080
서버를 중지하려면 Ctrl+C를 누르세요.
